In [8]:
import os
import pandas as pd

In [9]:
def get_data_file_path(current_trial_name="v2_eval_20_3", base_folder="game_logs", trial_base_name="alfworld_eval_", csv_file_name="alfworld_results.csv"):
    """returns the csv file path."""
    current_trial_folder = trial_base_name + current_trial_name
    file_path = os.path.join(base_folder, current_trial_folder, csv_file_name)
    return file_path

In [160]:
def convert_string_to_bool(in_string):
    """if string ="" False, else True"""
    if in_string:
        return True
    else:
        return False

def convert_string_to_bool_int(in_string):
    """if string ="" False, else True"""
    if in_string:
        return 1
    else:
        return 0

def convert_bool_string_to_bool_int(in_string):
    """ if string==False -> False, etc."""
    if in_string=="(False,)" or in_string=="False":
        return 0
    elif in_string=="(True,)" or in_string=="True":
        return 1
    else:
        print(type(in_string))
        print(in_string)
        print(in_string[0])
        raise Exception("Error in Transforming BoolString to Int")

def modify_text_prompt(keys_removed):
    """changes to short version"""
    keys = keys_removed.count("+")
    if keys==0:
        return keys_removed
    elif keys>1:
        return "short"
    else:
        return "long"

In [165]:
CURRENT_TRIAL_NAME = "v2_eval_20_3"
file_path = get_data_file_path(current_trial_name=CURRENT_TRIAL_NAME)
results_data = pd.read_csv(
    os.path.join("..",file_path), 
    converters = {
        "early_stop":convert_string_to_bool_int, 
        "error": convert_string_to_bool_int,
        "done": convert_bool_string_to_bool_int,
        "success": convert_bool_string_to_bool_int,
        "keys_removed": modify_text_prompt
    }
)

header_order = ["model", "keys_removed","temperature","env_type","success","done","early_stop","num_of_steps","num_nothing_happens","num_repetitions","error"]
clean_results = results_data[header_order]
clean_results.head()

,model,keys_removed,temperature,env_type,success,done,early_stop,num_of_steps,num_nothing_happens,num_repetitions,error
0,gpt-3.5-turbo-0125,react-1,0.0,cool,0,1,0,49,12,0,0
1,gpt-3.5-turbo-0125,react-1,0.0,clean,0,1,0,49,13,0,0
2,gpt-3.5-turbo-0125,react-1,0.0,clean,0,1,0,49,33,0,0
3,gpt-3.5-turbo-0125,react-1,0.0,put,0,1,0,49,26,0,0
4,gpt-3.5-turbo-0125,react-1,0.0,clean,0,1,0,49,39,2,0


In [168]:
out = clean_results.groupby(["model","keys_removed","env_type"]).sum().reset_index()
out["total"] = clean_results.groupby(["model","keys_removed","env_type"])["success"].count().reset_index()["success"]
out_header_order = ["model", "keys_removed","temperature","env_type","total","success","done","early_stop","num_of_steps","num_nothing_happens","num_repetitions"]
out = out[out_header_order]

In [169]:
x = out.groupby(["env_type"])
x.head()

,model,keys_removed,temperature,env_type,total,success,done,early_stop,num_of_steps,num_nothing_happens,num_repetitions
0,gpt-3.5-turbo-0125,agentbench-v2,0.0,clean,4,0,0,4,56,30,20
1,gpt-3.5-turbo-0125,agentbench-v2,0.0,cool,5,0,2,3,126,64,15
2,gpt-3.5-turbo-0125,agentbench-v2,0.0,examine,2,1,2,0,64,26,1
3,gpt-3.5-turbo-0125,agentbench-v2,0.0,heat,3,0,2,1,115,65,9
4,gpt-3.5-turbo-0125,agentbench-v2,0.0,put,6,2,3,3,170,81,32
5,gpt-3.5-turbo-0125,long,0.0,clean,4,2,4,0,116,65,1
6,gpt-3.5-turbo-0125,long,0.0,cool,5,1,3,2,178,80,20
7,gpt-3.5-turbo-0125,long,0.0,examine,2,0,1,1,69,58,5
8,gpt-3.5-turbo-0125,long,0.0,heat,3,2,2,1,28,9,6
9,gpt-3.5-turbo-0125,long,0.0,put,6,0,2,4,203,90,22
